In [529]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.mllib.recommendation import ALS as newALS

In [530]:
ratings_path = "~/Downloads/ml-100k/u.data"
df = pd.read_csv(ratings_path, sep = '\t').sort_values('timestamp')

In [531]:
#ratings_path = "~/Downloads/ml-1m/ratings.dat"
#df = pd.read_csv(ratings_path, sep = '::', names=['user_id', 'movie_id', 'rating', 'timestamp']).sort_values('timestamp')

In [532]:
df

,user_id,movie_id,rating,timestamp
214,259,255,4,874724710
83965,259,286,4,874724727
43027,259,298,4,874724754
21396,259,185,4,874724781
82655,259,173,4,874724843
...,...,...,...,...
46773,729,689,4,893286638
73008,729,313,3,893286638
46574,729,328,3,893286638
64312,729,748,4,893286638


In [533]:
len(df)

100000

In [534]:
squash_amount = 100

In [535]:
new_timestamp = [i // squash_amount for i in range(1, len(df)+1)]
new_timestamp

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [536]:
new_df = df.drop('timestamp', 1)
new_df['timestamp'] = new_timestamp

/var/folders/p5/zwk89rgd0cdf07_7k1wdqjgw0000gn/T/ipykernel_56430/405703479.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  new_df = df.drop('timestamp', 1)


In [537]:
new_df

,user_id,movie_id,rating,timestamp
214,259,255,4,0
83965,259,286,4,0
43027,259,298,4,0
21396,259,185,4,0
82655,259,173,4,0
...,...,...,...,...
46773,729,689,4,999
73008,729,313,3,999
46574,729,328,3,999
64312,729,748,4,999


In [538]:
def train_stream_test_split(df):
    columns = ["user_id", "movie_id", "rating", "timestamp"]
    als_data = []
    stream_data = []
    for user_id in df["user_id"].unique():
        user_df = df[df["user_id"] == user_id].sort_values("timestamp")
        count = 0
        for row in user_df.itertuples():
            data = [row.user_id, row.movie_id, row.rating, row.timestamp]
            if count < len(user_df) / 2:
                als_data.append(data)
            else:
                stream_data.append(data)
            count += 1
    als_df = pd.DataFrame(data=als_data, columns=columns).sort_values("timestamp")
    movie_list = set(als_df['movie_id'])
    stream_df = pd.DataFrame(data=stream_data, columns=columns).sort_values("timestamp")
    # Drop movies not seen before from stream
    stream_df = stream_df.drop(stream_df[stream_df['movie_id'].map(lambda x: x not in movie_list)].index)
    test_df = stream_df.copy()
    return als_df, stream_df, test_df

In [539]:
als_df, stream_df, test_df = train_stream_test_split(new_df)

In [540]:
spark.stop()

In [541]:
spark = SparkSession.builder.master('local').appName('als').getOrCreate()

In [542]:
als_df = als_df.drop("timestamp", 1)

/var/folders/p5/zwk89rgd0cdf07_7k1wdqjgw0000gn/T/ipykernel_56430/2193871712.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  als_df = als_df.drop("timestamp", 1)


In [543]:
spark_als_df = spark.createDataFrame(als_df) 
spark_stream_df = spark.createDataFrame(stream_df)

In [544]:
spark_als_df.toPandas()

,user_id,movie_id,rating
0,259,255,4
1,712,83,4
2,712,196,4
3,712,191,3
4,712,716,5
...,...,...,...
50235,683,1483,3
50236,683,22,4
50237,683,626,3
50238,683,62,4


In [545]:
als = ALS(
         userCol="user_id", 
         itemCol="movie_id",
         ratingCol="rating", 
         nonnegative = True, 
         implicitPrefs = False,
         coldStartStrategy="drop",
         rank=150,
         maxIter=10,
         regParam=.1
)
model=als.fit(spark_als_df)
predictions = model.transform(spark_stream_df)
evaluator = RegressionEvaluator().setMetricName("rmse").setLabelCol("rating").setPredictionCol("prediction")
rmse = evaluator.evaluate(predictions)
rmse

0.9806914059604086

In [546]:
new_model = newALS.train(ratings = spark_als_df.rdd, rank = 150, iterations=15, lambda_=.2)

In [547]:
new_model.userFeatures().toDF().toPandas()

,_1,_2
0,1,"[0.14272034168243408, -0.054845117032527924, -..."
1,2,"[0.14554466307163239, -0.03576328605413437, -0..."
2,3,"[0.12067998200654984, -0.13371162116527557, -0..."
3,4,"[0.14696075022220612, -0.11699432879686356, -0..."
4,5,"[0.1312849372625351, -0.0839436799287796, -0.1..."
...,...,...
938,939,"[0.11880175769329071, -0.06845995783805847, -0..."
939,940,"[0.04993217810988426, -0.10215268284082413, -0..."
940,941,"[0.11588550359010696, -0.1067308709025383, -0...."
941,942,"[0.15945583581924438, -0.08291233330965042, -0..."


In [548]:
user_features = new_model.userFeatures().toDF().toPandas()
user_features = user_features.rename(columns={'_1': 'id', '_2': 'features'})
user_features

22/02/22 19:28:48 WARN BlockManager: Task 315 already completed, not releasing lock for rdd_547_0


,id,features
0,1,"[0.14272034168243408, -0.054845117032527924, -..."
1,2,"[0.14554466307163239, -0.03576328605413437, -0..."
2,3,"[0.12067998200654984, -0.13371162116527557, -0..."
3,4,"[0.14696075022220612, -0.11699432879686356, -0..."
4,5,"[0.1312849372625351, -0.0839436799287796, -0.1..."
...,...,...
938,939,"[0.11880175769329071, -0.06845995783805847, -0..."
939,940,"[0.04993217810988426, -0.10215268284082413, -0..."
940,941,"[0.11588550359010696, -0.1067308709025383, -0...."
941,942,"[0.15945583581924438, -0.08291233330965042, -0..."


In [549]:
product_features = new_model.productFeatures().toDF().toPandas()
product_features = product_features.rename(columns={'_1': 'id', '_2': 'features'})

22/02/22 19:28:49 WARN BlockManager: Task 317 already completed, not releasing lock for rdd_548_0


In [550]:
dir_path = "~/movielens/ralf/"

In [551]:
product_features.sort_values('id').to_csv(dir_path + 'movie_features.csv', index = False)

In [552]:
user_features.sort_values('id').to_csv(dir_path + 'user_features.csv', index = False)

In [553]:
stream_df = stream_df.sample(frac=1)
new_timestamp = [i // squash_amount for i in range(1, len(stream_df)+1)]
stream_df['timestamp'] = new_timestamp

In [554]:
stream_df

,user_id,movie_id,rating,timestamp
48981,796,546,4,0
36754,110,325,3,0
7149,711,354,3,0
22887,795,17,2,0
15663,843,1411,3,0
...,...,...,...,...
4167,268,1413,2,494
48266,280,66,5,494
7710,182,50,5,494
29112,487,1188,3,494


In [555]:
print(len(stream_df))

49442


In [556]:
stream_df.to_csv(dir_path + 'ratings.csv', index = False)

22/02/22 22:35:50 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 482335 ms exceeds timeout 120000 ms
22/02/22 22:35:50 WARN SparkContext: Killing executors is not supported by current scheduler.
